<a href="https://colab.research.google.com/github/thanospapastef/Simulations-from-a-mixture-model/blob/main/Simulations_from_a_mixture_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.stats import norm
import numpy as  np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
import pymc3 as pm
import arviz as az
import time
tfd = tfp.distributions

In [2]:
#1st experiment
sigma2 = 0.15

In [ ]:
#Gibbs Sampler
N = 100_000
start = time.time()
normal1= np.random.normal(-1, np.sqrt(sigma2), int(N/2))
normal2 = np.random.normal(1, np.sqrt(sigma2), int(N/2))
target = np.concatenate([normal1, normal2])

it2 = []
for i in range(0, N):
  it2.append(i)

end = time.time()
print(f"Time elapsed for Gibbs sampler: {end-start} seconds")

#plots
plt.subplots(figsize=(20, 5))
plt.subplot(1,2,1)
sns.distplot(target, hist = False )

plt.subplot(1,2,2)
plt.plot(it2, target)
plt.title('Chain')

plt.show()

In [ ]:
# Metropolis - Hastings method
# Define mixture model
N = 100_000

weights = np.array([0.5, 0.5])
means = np.array([-1, 1])
deviations = np.array([np.sqrt(sigma2), np.sqrt(sigma2)])

start = time.time()

basic_model = pm.Model()

with basic_model:
  Y_obs = pm.distributions.mixture.NormalMixture('Metropolis - Hastings', w = weights, mu = means, sigma = deviations)

with basic_model:
    # define sampler
    step = pm.Metropolis()

    # simulating N samples
    trace = pm.sample(draws=N,  return_inferencedata = True, step=step, chains = 1)
    
end = time.time()
print(f"Time elapsed for Metropolis - Hastings sampler: {end-start} seconds")

In [ ]:
with basic_model:
    az.plot_trace(trace);

In [ ]:
az.ess(trace) #effective_sample_size

In [ ]:
# Hamiltonian Monte Carlo method
# Define mixture model
N = 100_000

weights = np.array([0.5, 0.5])
means = np.array([-1, 1])
deviations = np.array([np.sqrt(sigma2), np.sqrt(sigma2)])

start = time.time()

basic_model = pm.Model()

with basic_model:
  Y_obs = pm.distributions.mixture.NormalMixture('Hamiltonian Monte Carlo', w = weights, mu = means, sigma = deviations)

with basic_model:
    # define sampler
    step = pm.HamiltonianMC()

    # simulating N samples
    trace = pm.sample(draws=N, return_inferencedata = True, step=step, chains = 1)
    
end = time.time()
print(f"Time elapsed for Hamiltonian Monte Carlo sampler: {end-start} seconds")

In [ ]:
import arviz as az
with basic_model:
    az.plot_trace(trace);

In [ ]:
az.ess(trace) #effective_sample_size

In [ ]:
#Tempering method
dtype = np.float32
N = 100_000

target = tfd.MixtureSameFamily(
    mixture_distribution=tfd.Categorical(
        probs=[0.5, 0.5]),
    components_distribution=tfd.Normal(
      loc=[-1., 1],       # One for each component.
      scale=[np.sqrt(sigma2).astype(dtype), np.sqrt(sigma2).astype(dtype)])) 

#9 Different "temperatures"
inverse_temperatures = 0.2**tf.range(9, dtype=dtype)

def make_kernel_fn(target_log_prob_fn):   
   return tfp.mcmc.RandomWalkMetropolis(
    target_log_prob_fn, new_state_fn=None,
    name=None) 

start = time.time()

#ReplicaExchangeMC == Parallel Tempering
remc = tfp.mcmc.ReplicaExchangeMC(
    target_log_prob_fn=target.log_prob,
    inverse_temperatures=inverse_temperatures,
    make_kernel_fn=make_kernel_fn)

samples = tfp.mcmc.sample_chain(
    num_results=N,
    current_state = 4.0,
    kernel=remc,
    trace_fn=None,
    num_burnin_steps=500)

#iterations
it1 = []
for i in range(0, N):
  it1.append(i)

end = time.time()

print(f"Time elapsed for Tempering sampler: {end-start} seconds")

plt.subplots(figsize=(20, 5))
#plots
plt.subplot(1,2,1)
sns.distplot(samples, hist = False )

plt.subplot(1,2,2)
plt.plot(it1, samples)
plt.title('Chain')
  
# Packing all the plots and displaying them
plt.tight_layout()
plt.show()

In [11]:
#2nd experiment
sigma2 = 0.2

In [ ]:
#Gibbs Sampler
N = 100_000
start = time.time()
normal1= np.random.normal(-1, np.sqrt(sigma2), int(N/2))
normal2 = np.random.normal(1, np.sqrt(sigma2), int(N/2))
target = np.concatenate([normal1, normal2])

it2 = []
for i in range(0, N):
  it2.append(i)

end = time.time()
print(f"Time elapsed for Gibbs sampler: {end-start} seconds")

#plots
plt.subplots(figsize=(20, 5))
plt.subplot(1,2,1)
sns.distplot(target, hist = False )

plt.subplot(1,2,2)
plt.plot(it2, target)
plt.title('Chain')

plt.show()

In [ ]:
#Metropolis - Hastings
# Define mixture model
N = 100_000

weights = np.array([0.5, 0.5])
means = np.array([-1, 1])
deviations = np.array([np.sqrt(sigma2), np.sqrt(sigma2)])

start = time.time()

basic_model = pm.Model()

with basic_model:
  Y_obs = pm.distributions.mixture.NormalMixture('Metropolis - Hastings', w = weights, mu = means, sigma = deviations)

with basic_model:
    # Define sampler
    step = pm.Metropolis()

    # Simulating N samples
    trace = pm.sample(draws=N,  return_inferencedata = True, step=step, chains = 1)
    
end = time.time()
print(f"Time elapsed for Metropolis - Hastings sampler: {end-start} seconds")

In [ ]:
import arviz as az
with basic_model:
    az.plot_trace(trace);

In [ ]:
az.ess(trace) #effective_sample_size

In [ ]:
#Hamiltonian Monte Carlo method
# Define mixture model
N = 100_000


weights = np.array([0.5, 0.5])
means = np.array([-1, 1])
deviations = np.array([np.sqrt(sigma2), np.sqrt(sigma2)])

start = time.time()

basic_model = pm.Model()

with basic_model:
  Y_obs = pm.distributions.mixture.NormalMixture('Hamiltonian Monte Carlo', w = weights, mu = means, sigma = deviations)

with basic_model:
    # Define sampler
    step = pm.HamiltonianMC()

    # Simulating N samples
    trace = pm.sample(draws=N, return_inferencedata = True, step=step, chains = 1)
    
end = time.time()
print(f"Time elapsed for Hamiltonian Monte Carlo sampler: {end-start} seconds")

In [ ]:
import arviz as az
with basic_model:
    az.plot_trace(trace);

In [ ]:
az.ess(trace) #effective_sample_size

In [ ]:
#Tempering method
dtype = np.float32
N = 100_000

target = tfd.MixtureSameFamily(
    mixture_distribution=tfd.Categorical(
        probs=[0.5, 0.5]),
    components_distribution=tfd.Normal(
      loc=[-1., 1],       # One for each component.
      scale=[np.sqrt(sigma2).astype(dtype), np.sqrt(sigma2).astype(dtype)]))  # And same here.

#9 Different "temperatures"
inverse_temperatures = 0.2**tf.range(9, dtype=dtype)

def make_kernel_fn(target_log_prob_fn):   
   return tfp.mcmc.RandomWalkMetropolis(
    target_log_prob_fn, new_state_fn=None,
    name=None) 

start = time.time()

#ReplicaExchangeMC == Parallel Tempering
remc = tfp.mcmc.ReplicaExchangeMC(
    target_log_prob_fn=target.log_prob,
    inverse_temperatures=inverse_temperatures,
    make_kernel_fn=make_kernel_fn)

samples = tfp.mcmc.sample_chain(
    num_results=N,
    current_state = 4.0,
    kernel=remc,
    trace_fn=None,
    num_burnin_steps=500)

#iterations
it1 = []
for i in range(0, N):
  it1.append(i)

end = time.time()

print(f"Time elapsed for Tempering sampler: {end-start} seconds")

plt.subplots(figsize=(20, 5))
#plots
plt.subplot(1,2,1)
sns.distplot(samples, hist = False )

plt.subplot(1,2,2)
plt.plot(it1, samples)
plt.title('Chain')
  
# Packing all the plots and displaying them
plt.tight_layout()
plt.show()

In [ ]:
#3rd experiment
sigma2 = 0.25

In [ ]:
#Gibbs Sampler
N = 100_000
start = time.time()
normal1= np.random.normal(-1, np.sqrt(sigma2), int(N/2))
normal2 = np.random.normal(1, np.sqrt(sigma2), int(N/2))
target = np.concatenate([normal1, normal2])

it2 = []
for i in range(0, N):
  it2.append(i)

end = time.time()
print(f"Time elapsed for Gibbs sampler: {end-start} seconds")

#plots
plt.subplots(figsize=(20, 5))
plt.subplot(1,2,1)
sns.distplot(target, hist = False )

plt.subplot(1,2,2)
plt.plot(it2, target)
plt.title('Chain')

plt.show()

In [ ]:
#Metropolis - Hastings
# Define mixture model
N = 100_000

weights = np.array([0.5, 0.5])
means = np.array([-1, 1])
deviations = np.array([np.sqrt(sigma2), np.sqrt(sigma2)])

start = time.time()

basic_model = pm.Model()

with basic_model:
  Y_obs = pm.distributions.mixture.NormalMixture('Metropolis - Hastings', w = weights, mu = means, sigma = deviations)

with basic_model:
    # Define sampler
    step = pm.Metropolis()

    # Simulating N samples
    trace = pm.sample(draws=N,  return_inferencedata = True, step=step, chains = 1)
    
end = time.time()
print(f"Time elapsed for Metropolis - Hastings sampler: {end-start} seconds")

In [ ]:
import arviz as az
with basic_model:
    az.plot_trace(trace);

In [ ]:
az.ess(trace) #effective_sample_size

In [ ]:
#Hamiltonian Monte Carlo method
# Define mixture model
N = 100_000

weights = np.array([0.5, 0.5])
means = np.array([-1, 1])
deviations = np.array([np.sqrt(sigma2), np.sqrt(sigma2)])

start = time.time()

basic_model = pm.Model()

with basic_model:
  Y_obs = pm.distributions.mixture.NormalMixture('Hamiltonian Monte Carlo', w = weights, mu = means, sigma = deviations)

with basic_model:
    # Define sampler
    step = pm.HamiltonianMC()

    # Simulating N samples
    trace = pm.sample(draws=N, return_inferencedata = True, step=step, chains = 1)
    
end = time.time()
print(f"Time elapsed for Hamiltonian Monte Carlo sampler: {end-start} seconds")

In [ ]:
import arviz as az
with basic_model:
    az.plot_trace(trace);

In [ ]:
az.ess(trace) #effective_sample_size

In [ ]:
#Tempering method
dtype = np.float32
N = 100_000

target = tfd.MixtureSameFamily(
    mixture_distribution=tfd.Categorical(
        probs=[0.5, 0.5]),
    components_distribution=tfd.Normal(
      loc=[-1., 1],       # One for each component.
      scale=[np.sqrt(sigma2).astype(dtype), np.sqrt(sigma2).astype(dtype)]))  # And same here.

#9 Different "temperatures"
inverse_temperatures = 0.2**tf.range(9, dtype=dtype)

def make_kernel_fn(target_log_prob_fn):   
   return tfp.mcmc.RandomWalkMetropolis(
    target_log_prob_fn, new_state_fn=None,
    name=None) 

start = time.time()

#ReplicaExchangeMC == Parallel Tempering
remc = tfp.mcmc.ReplicaExchangeMC(
    target_log_prob_fn=target.log_prob,
    inverse_temperatures=inverse_temperatures,
    make_kernel_fn=make_kernel_fn)

samples = tfp.mcmc.sample_chain(
    num_results=N,
    current_state = 4.0,
    kernel=remc,
    trace_fn=None,
    num_burnin_steps=500)

#iterations
it1 = []
for i in range(0, N):
  it1.append(i)

end = time.time()

print(f"Time elapsed for Tempering sampler: {end-start} seconds")

plt.subplots(figsize=(20, 5))
#plots
plt.subplot(1,2,1)
sns.distplot(samples, hist = False )

plt.subplot(1,2,2)
plt.plot(it1, samples)
plt.title('Chain')
  
# Packing all the plots and displaying them
plt.tight_layout()
plt.show()